<a href="https://colab.research.google.com/github/Tasfia-007/QOSF-Mentorship-Screeing_Tasks/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Qiskit and qiskit-aer if not already installed
!pip install qiskit --upgrade
!pip install qiskit-aer --upgrade

# Importing essential libraries from Qiskit and numpy
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

def add_noise(prob_1q: float, prob_2q: float, circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Add random Pauli noise to a quantum circuit based on specified probabilities.

    Parameters:
    - prob_1q: Probability of applying a random Pauli operator after a single-qubit gate.
    - prob_2q: Probability of applying a random Pauli operator after a two-qubit gate.
    - circuit: The quantum circuit where the noise will be introduced.

    Returns:
    - A new quantum circuit with noise applied.
    """

    # Create a new circuit to apply the noise to, including the classical registers for measurement
    noisy_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs)

    # Define the Pauli operators: Identity (I), X, Y, Z
    pauli_ops = ['I', 'X', 'Y', 'Z']

    # Loop through the gates in the original circuit using the updated Qiskit standard
    for instr in circuit.data:
        operation = instr.operation  # Get the operation (e.g., gate or measurement)
        qargs = instr.qubits  # Get the qubits involved in the operation
        cargs = instr.clbits  # Get the classical bits, if any

        # Append the original operation to the new noisy circuit
        noisy_circuit.append(operation, qargs, cargs)

        # If it's a single-qubit gate, apply noise with a given probability
        if operation.num_qubits == 1 and np.random.rand() < prob_1q:
            random_pauli = np.random.choice(pauli_ops[1:])  # Randomly pick X, Y, or Z (excluding I)
            if random_pauli == 'X':
                noisy_circuit.x(qargs[0])  # Apply X gate
            elif random_pauli == 'Y':
                noisy_circuit.y(qargs[0])  # Apply Y gate
            elif random_pauli == 'Z':
                noisy_circuit.z(qargs[0])  # Apply Z gate

        # If it's a two-qubit gate, apply noise to both qubits with the given probability
        elif operation.num_qubits == 2 and np.random.rand() < prob_2q:
            for qubit in qargs:
                random_pauli = np.random.choice(pauli_ops[1:])  # Randomly pick X, Y, or Z (excluding I)
                if random_pauli == 'X':
                    noisy_circuit.x(qubit)  # Apply X gate to the qubit
                elif random_pauli == 'Y':
                    noisy_circuit.y(qubit)  # Apply Y gate to the qubit
                elif random_pauli == 'Z':
                    noisy_circuit.z(qubit)  # Apply Z gate to the qubit

    # Return the newly created circuit with noise added
    return noisy_circuit

# Creating a simple 2-qubit circuit to test the noise function
qc = QuantumCircuit(2, 2)  # Quantum circuit with 2 qubits and 2 classical bits
qc.h(0)   # Hadamard gate on qubit 0, putting it into superposition
qc.cx(0, 1)  # CNOT gate entangling qubits 0 and 1

# Add measurement gates to both qubits
qc.measure([0, 1], [0, 1])

# Set probabilities for adding noise after single-qubit and two-qubit gates
prob_1q = 0.1  # 10% chance of noise after a single-qubit gate
prob_2q = 0.2  # 20% chance of noise after a two-qubit gate

# Create a noisy version of the circuit
noisy_qc = add_noise(prob_1q, prob_2q, qc)

# Display the noisy circuit to see the effect of adding noise
print(noisy_qc)

# Optional: Simulate the noisy circuit using Qiskit's Aer simulator
simulator = AerSimulator()
compiled_noisy_circuit = transpile(noisy_qc, simulator)  # Transpile the circuit to run on the simulator
result = simulator.run(compiled_noisy_circuit, shots=1024).result()  # Run the simulation with 1024 shots

# Get and print the measurement results
counts = result.get_counts()
print(counts)


     ┌───┐     ┌─┐        
q_0: ┤ H ├──■──┤M├────────
     └───┘┌─┴─┐└╥┘┌─┐┌───┐
q_1: ─────┤ X ├─╫─┤M├┤ Y ├
          └───┘ ║ └╥┘└───┘
c: 2/═══════════╩══╩══════
                0  1      
{'00': 522, '11': 502}


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def transform_to_gate_basis(circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Converts a quantum circuit to use only the specified gate set: {CX, ID, RZ, SX, X}.

    Parameters:
    - circuit: The quantum circuit to be transformed.

    Returns:
    - A new quantum circuit that only uses gates from the specified gate set.
    """
    # Define the set of gates that we want to allow in the transformed circuit
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']

    # Use Qiskit's transpile function to convert the circuit to only use these gates
    transformed_circuit = transpile(circuit, basis_gates=basis_gates)

    return transformed_circuit

# Create a quantum circuit with a variety of gates for demonstration
qc = QuantumCircuit(2)
qc.h(0)        # Hadamard gate on qubit 0 (this will need to be converted to the allowed gates)
qc.cx(0, 1)    # Controlled-X (CX) gate between qubit 0 and qubit 1 (this gate is already in the allowed set)
qc.rz(1.57, 1) # RZ gate on qubit 1 (already in the allowed set)
qc.x(0)        # X gate on qubit 0 (already in the allowed set)
qc.sx(1)       # SX gate on qubit 1 (already in the allowed set)

# Add measurements to the circuit, which measure the qubits and store results in classical bits
qc.measure_all()

# Now transform the circuit so that it only uses the allowed gate set {CX, ID, RZ, SX, X}
transformed_qc = transform_to_gate_basis(qc)

# Print out both the original and transformed circuits so we can compare them
print("Original Circuit:")
print(qc)
print("\nTransformed Circuit (using only CX, ID, RZ, SX, X):")
print(transformed_qc)

# Optionally, we can simulate this transformed circuit to see how it behaves
simulator = AerSimulator()
compiled_transformed_qc = transpile(transformed_qc, simulator)  # Transpile again for the specific simulator
result = simulator.run(compiled_transformed_qc, shots=1024).result()  # Run the simulation with 1024 shots

# Get the measurement results (counts) and display them
counts = result.get_counts()
print("\nSimulation Results (Measurement Counts):")
print(counts)


Original Circuit:
        ┌───┐        ┌───┐           ░ ┌─┐   
   q_0: ┤ H ├──■─────┤ X ├───────────░─┤M├───
        └───┘┌─┴─┐┌──┴───┴───┐┌────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Rz(1.57) ├┤ √X ├─░──╫─┤M├
             └───┘└──────────┘└────┘ ░  ║ └╥┘
meas: 2/════════════════════════════════╩══╩═
                                        0  1 

Transformed Circuit (using only CX, ID, RZ, SX, X):
global phase: π/4
        ┌─────────┐┌────┐┌─────────┐        ┌───┐           ░ ┌─┐   
   q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■─────┤ X ├───────────░─┤M├───
        └─────────┘└────┘└─────────┘┌─┴─┐┌──┴───┴───┐┌────┐ ░ └╥┘┌─┐
   q_1: ────────────────────────────┤ X ├┤ Rz(1.57) ├┤ √X ├─░──╫─┤M├
                                    └───┘└──────────┘└────┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════════════════════════╩══╩═
                                                               0  1 

Simulation Results (Measurement Counts):
{'00': 224, '10': 264, '01': 262, '11': 274}


In [4]:
from qiskit import QuantumCircuit
import numpy as np

def qft(circuit, n):
    """
    Applies the Quantum Fourier Transform (QFT) to the first n qubits in the circuit.

    Parameters:
    - circuit: The quantum circuit where QFT will be applied.
    - n: The number of qubits to apply the QFT on.
    """
    for i in range(n):
        # Step 1: Apply a Hadamard gate to the current qubit
        circuit.h(i)

        # Step 2: Apply controlled phase shifts between the current qubit and all qubits after it
        for j in range(i + 1, n):
            # The angle of the controlled phase shift gets smaller with increasing distance between qubits
            circuit.cp(np.pi / (2 ** (j - i)), i, j)

    # Step 3: Swap qubits to reverse their order (necessary for QFT)
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

def inverse_qft(circuit, n):
    """
    Applies the inverse Quantum Fourier Transform (QFT†) to the first n qubits in the circuit.

    Parameters:
    - circuit: The quantum circuit where the inverse QFT will be applied.
    - n: The number of qubits to apply the inverse QFT on.
    """
    # Step 1: Swap the qubits to reverse their order, undoing the swaps from the original QFT
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

    # Step 2: Apply the inverse controlled phase shifts and Hadamard gates in reverse order
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            # The inverse of a phase shift is just a negative phase shift
            circuit.cp(-np.pi / (2 ** (j - i)), i, j)
        circuit.h(i)

def quantum_sum(a: int, b: int, num_bits: int) -> QuantumCircuit:
    """
    Implements the Draper adder algorithm to add two integers using quantum registers and the QFT.

    Parameters:
    - a: The first integer to add.
    - b: The second integer to add.
    - num_bits: The number of qubits required to represent the integers.

    Returns:
    - A quantum circuit that adds the two integers using the Draper Adder algorithm.
    """
    # Create a quantum circuit with enough qubits to represent both integers and their sum
    qc = QuantumCircuit(2 * num_bits)

    # Convert 'a' and 'b' to their binary representation, padded to the number of bits
    a_bin = bin(a)[2:].zfill(num_bits)
    b_bin = bin(b)[2:].zfill(num_bits)

    # Step 1: Initialize the first register to the binary value of 'a'
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i)  # Apply X gate to set qubits to the value of 'a'

    # Step 2: Initialize the second register to the binary value of 'b'
    for i, bit in enumerate(reversed(b_bin)):
        if bit == '1':
            qc.x(num_bits + i)  # Apply X gate to set qubits to the value of 'b'

    # Step 3: Apply the Quantum Fourier Transform (QFT) to the first set of qubits (where 'a' is stored)
    qft(qc, num_bits)

    # Step 4: Perform controlled phase rotations between the 'a' qubits and 'b' qubits
    for i in range(num_bits):
        for j in range(i, num_bits):
            # The angle of rotation depends on the distance between the qubits
            angle = np.pi / (2 ** (j - i + 1))
            qc.cp(angle, num_bits + j, i)  # Controlled phase rotation from qubits in 'b' to qubits in 'a'

    # Step 5: Apply the inverse QFT to the first set of qubits to complete the addition
    inverse_qft(qc, num_bits)

    # Return the completed Draper Adder circuit
    return qc

# Example usage:
# We want to add two numbers, a = 3 and b = 2, using the Draper Adder

a = 3  # First number to add
b = 2  # Second number to add
num_bits = 3  # Number of bits needed to represent the numbers (at least 3 to avoid overflow)

# Build the Draper Adder circuit for the sum of a and b
qc = quantum_sum(a, b, num_bits)

# Print the quantum circuit
print("Quantum Circuit for Draper Adder to add", a, "and", b)
print(qc)


Quantum Circuit for Draper Adder to add 3 and 2
     ┌───┐┌───┐                                                          »
q_0: ┤ X ├┤ H ├─■────────■───────────────────────────X──■────────■───────»
     ├───┤└───┘ │P(π/2)  │       ┌───┐               │  │        │       »
q_1: ┤ X ├──────■────────┼───────┤ H ├─■─────────────┼──┼────────┼───────»
     └───┘               │P(π/4) └───┘ │P(π/2) ┌───┐ │  │        │       »
q_2: ────────────────────■─────────────■───────┤ H ├─X──┼────────┼───────»
                                               └───┘    │P(π/2)  │       »
q_3: ───────────────────────────────────────────────────■────────┼───────»
     ┌───┐                                                       │P(π/4) »
q_4: ┤ X ├───────────────────────────────────────────────────────■───────»
     └───┘                                                               »
q_5: ────────────────────────────────────────────────────────────────────»
                                                    

In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
import numpy as np

# Quantum Fourier Transform (QFT) Function
def qft(circuit, n):
    """
    Applies the Quantum Fourier Transform (QFT) to the first n qubits in the circuit.
    """
    for i in range(n):
        # Apply a Hadamard gate to the current qubit
        circuit.h(i)
        # Apply controlled phase shifts to all qubits after the current qubit
        for j in range(i + 1, n):
            circuit.cp(np.pi / (2 ** (j - i)), i, j)

    # Swap the qubits to reverse their order for QFT
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

# Inverse Quantum Fourier Transform (QFT†)
def inverse_qft(circuit, n):
    """
    Applies the inverse Quantum Fourier Transform (QFT†) to the first n qubits in the circuit.
    """
    # Reverse the qubit order to undo the QFT swaps
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

    # Apply the inverse phase shifts and Hadamard gates
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            circuit.cp(-np.pi / (2 ** (j - i)), i, j)
        circuit.h(i)

# Draper Adder to perform quantum addition using QFT
def quantum_sum(a: int, b: int, num_bits: int):
    """
    Implements the Draper Adder algorithm to add two integers using quantum registers and QFT.

    Parameters:
    - a: First integer to add.
    - b: Second integer to add.
    - num_bits: Number of qubits needed to represent the integers.

    Returns:
    - A quantum circuit that adds 'a' and 'b' using the Draper Adder.
    """
    qc = QuantumCircuit(2 * num_bits)

    # Convert the numbers to binary and pad them with zeros
    a_bin = bin(a)[2:].zfill(num_bits)
    b_bin = bin(b)[2:].zfill(num_bits)

    # Initialize the first register with the binary value of 'a'
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i)

    # Initialize the second register with the binary value of 'b'
    for i, bit in enumerate(reversed(b_bin)):
        if bit == '1':
            qc.x(num_bits + i)

    # Apply QFT to the first register
    qft(qc, num_bits)

    # Controlled phase rotations between 'a' and 'b'
    for i in range(num_bits):
        for j in range(i, num_bits):
            angle = np.pi / (2 ** (j - i + 1))
            qc.cp(angle, num_bits + j, i)

    # Apply inverse QFT to the first register
    inverse_qft(qc, num_bits)

    return qc

# Function to convert the circuit to a specific gate basis
def transform_to_gate_basis(circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Transforms the quantum circuit to use only gates from the set: {CX, ID, RZ, SX, X}.
    """
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
    return transpile(circuit, basis_gates=basis_gates)

# Function to add noise to the circuit
def add_noise(prob_1q: float, prob_2q: float, circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Adds noise to a quantum circuit by randomly applying Pauli errors after gates.

    Parameters:
    - prob_1q: Probability of applying noise after a single-qubit gate.
    - prob_2q: Probability of applying noise after a two-qubit gate.
    - circuit: The quantum circuit to which noise will be added.

    Returns:
    - A new circuit with noise applied.
    """
    noisy_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs)
    pauli_ops = ['I', 'X', 'Y', 'Z']

    for instr in circuit.data:
        operation = instr.operation
        qubits = instr.qubits
        clbits = instr.clbits

        # Append the original operation
        noisy_circuit.append(operation, qubits, clbits)

        # Apply noise based on probability
        if operation.num_qubits == 1 and np.random.rand() < prob_1q:
            random_pauli = np.random.choice(pauli_ops[1:])
            if random_pauli == 'X':
                noisy_circuit.x(qubits[0])
            elif random_pauli == 'Y':
                noisy_circuit.y(qubits[0])
            elif random_pauli == 'Z':
                noisy_circuit.z(qubits[0])
        elif operation.num_qubits == 2 and np.random.rand() < prob_2q:
            for qubit in qubits:
                random_pauli = np.random.choice(pauli_ops[1:])
                if random_pauli == 'X':
                    noisy_circuit.x(qubit)
                elif random_pauli == 'Y':
                    noisy_circuit.y(qubit)
                elif random_pauli == 'Z':
                    noisy_circuit.z(qubit)

    return noisy_circuit

# Function to simulate and analyze quantum addition with noise
def analyze_quantum_addition(a, b, num_bits, prob_1q, prob_2q):
    """
    Simulates the Draper Adder with noise, applying different noise levels and analyzing the results.
    """
    # Create the Draper Adder circuit
    qc = quantum_sum(a, b, num_bits)

    # Convert the circuit to the gate basis
    transformed_qc = transform_to_gate_basis(qc)

    # Add noise to the circuit
    noisy_qc = add_noise(prob_1q, prob_2q, transformed_qc)

    # Add measurement to all qubits
    noisy_qc.measure_all()

    # Simulate the circuit using AerSimulator
    simulator = AerSimulator()
    compiled_noisy_circuit = transpile(noisy_qc, simulator)
    result = simulator.run(compiled_noisy_circuit, shots=1024).result()

    # Print the measurement results
    counts = result.get_counts()
    print(f"\nResults for a = {a}, b = {b} with noise (1-qubit noise={prob_1q}, 2-qubit noise={prob_2q}):")
    print(counts)

# Run analysis with different levels of noise
a = 3
b = 2
num_bits = 3

# Analyze without noise
print("No noise:")
analyze_quantum_addition(a, b, num_bits, 0, 0)

# Analyze with low noise
print("\nLow noise:")
analyze_quantum_addition(a, b, num_bits, 0.01, 0.05)

# Analyze with high noise
print("\nHigh noise:")
analyze_quantum_addition(a, b, num_bits, 0.1, 0.2)


No noise:

Results for a = 3, b = 2 with noise (1-qubit noise=0, 2-qubit noise=0):
{'010110': 19, '010000': 219, '010011': 414, '010001': 223, '010111': 149}

Low noise:

Results for a = 3, b = 2 with noise (1-qubit noise=0.01, 2-qubit noise=0.05):
{'010110': 22, '010000': 227, '010001': 234, '010011': 432, '010111': 109}

High noise:

Results for a = 3, b = 2 with noise (1-qubit noise=0.1, 2-qubit noise=0.2):
{'101111': 14, '101001': 441, '101101': 59, '101110': 63, '101000': 436, '101100': 11}
